In [15]:
import numpy as np
from scipy import optimize
from scipy.io import loadmat

In [16]:
movie_ids = 'movie_ids.txt'
movies_mat = 'movies.mat'

In [17]:
def normalizeRatings(Y, R):
    m, n = Y.shape
    Ymean = np.zeros(m)
    Ynorm = np.zeros(Y.shape)

    for i in range(m):
        idx = R[i, :] == 1
        Ymean[i] = np.mean(Y[i, idx])
        Ynorm[i, idx] = Y[i, idx] - Ymean[i]

    return Ynorm, Ymean


def loadMovieList():
    with open(movie_ids,  encoding='ISO-8859-1') as fid:
        movies = fid.readlines()

    movieNames = []
    for movie in movies:
        parts = movie.split()
        movieNames.append(' '.join(parts[1:]).strip())
    return movieNames


def computeNumericalGradient(J, theta, e=1e-4):
    numgrad = np.zeros(theta.shape)
    perturb = np.diag(e * np.ones(theta.shape))
    for i in range(theta.size):
        loss1, _ = J(theta - perturb[:, i])
        loss2, _ = J(theta + perturb[:, i])
        numgrad[i] = (loss2 - loss1)/(2*e)
    return numgrad


def checkCostFunction(cofiCostFunc, lambda_=0.):
    X_t = np.random.rand(4, 3)
    Theta_t = np.random.rand(5, 3)

    Y = np.dot(X_t, Theta_t.T)
    Y[np.random.rand(*Y.shape) > 0.5] = 0
    R = np.zeros(Y.shape)
    R[Y != 0] = 1

    X = np.random.randn(*X_t.shape)
    Theta = np.random.randn(*Theta_t.shape)
    num_movies, num_users = Y.shape
    num_features = Theta_t.shape[1]

    params = np.concatenate([X.ravel(), Theta.ravel()])
    numgrad = computeNumericalGradient(
        lambda x: cofiCostFunc(x, Y, R, num_users, num_movies, num_features, lambda_), params)

    cost, grad = cofiCostFunc(params, Y, R, num_users,num_movies, num_features, lambda_)

    print(np.stack([numgrad, grad], axis=1))
    print('\nThe above two columns you get should be very similar.'
          '(Left-Your Numerical Gradient, Right-Analytical Gradient)')

    diff = np.linalg.norm(numgrad-grad)/np.linalg.norm(numgrad+grad)
    print('If your cost function implementation is correct, then '
          'the relative difference will be small (less than 1e-9).')
    print('\nRelative Difference: %g' % diff)

In [18]:
names = loadMovieList()

In [19]:
data = loadmat(movies_mat)
Y, R = data['Y'], data['R']

In [20]:
print('Average rating for movie ',names[1600],': %f / 5' %
      np.mean(Y[90, R[1600, :]]))

Average rating for movie  Office Killer (1997) : 4.994698 / 5


In [21]:
def cofiCostFunc(params, Y, R, num_users, num_movies,
                      num_features, lambda_=0.0):
    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)

    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)

    J = (1 / 2) * np.sum(np.square((X.dot(Theta.T) - Y) * R)) + (lambda_ / 2) * np.sum(np.square(X)) + \
                                                                (lambda_ / 2) * np.sum(np.square(Theta))
    
    for i in range(R.shape[0]):
        
        idx = np.where(R[i, :] == 1)[0]
        Theta_temp = Theta[idx, :]
        Y_temp = Y[i, idx]
        X_grad[i, :] = np.dot(np.dot(X[i, :], Theta_temp.T) - Y_temp, Theta_temp) + lambda_ * X[i, :]
        
    for j in range(R.shape[1]):
        
        idx = np.where(R[:, j] == 1)[0]
        X_temp = X[idx, :]
        Y_temp = Y[idx, j]
        Theta_grad[j, :] = np.dot(np.dot(X_temp, Theta[j, :]) - Y_temp, X_temp) + lambda_ * Theta[j, :]
    
    grad = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])
    return J, grad

In [22]:
checkCostFunction(cofiCostFunc, 1.5)

[[ 4.31229304  4.31229304]
 [ 2.21011136  2.21011136]
 [-3.99743875 -3.99743875]
 [-3.69038825 -3.69038825]
 [ 0.01903989  0.01903989]
 [-0.54902357 -0.54902357]
 [11.52397621 11.52397621]
 [-4.6158779  -4.6158779 ]
 [-9.6640715  -9.6640715 ]
 [-4.15396929 -4.15396929]
 [-5.86257022 -5.86257022]
 [ 3.95461942  3.95461942]
 [-0.25454828 -0.25454828]
 [ 3.84728211  3.84728211]
 [-1.54255876 -1.54255876]
 [ 0.36644666  0.36644666]
 [ 1.59381982  1.59381982]
 [ 1.23914717  1.23914717]
 [ 2.77990864  2.77990864]
 [ 4.33509101  4.33509101]
 [-2.07683109 -2.07683109]
 [12.98311824 12.98311824]
 [-7.70929635 -7.70929635]
 [-6.54471245 -6.54471245]
 [ 0.08799067  0.08799067]
 [ 0.9038655   0.9038655 ]
 [-0.23484719 -0.23484719]]

The above two columns you get should be very similar.(Left-Your Numerical Gradient, Right-Analytical Gradient)
If your cost function implementation is correct, then the relative difference will be small (less than 1e-9).

Relative Difference: 1.79014e-12


In [23]:
movieList = loadMovieList()

n_m = len(movieList)
my_ratings = np.zeros(n_m)

my_ratings[0] = 4
my_ratings[97] = 2
my_ratings[6] = 2
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63] = 5
my_ratings[364] = 2
my_ratings[68] = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 1
my_ratings[22] = 5
my_ratings[124] = 5
my_ratings[54] = 2

In [24]:

Y = np.hstack([my_ratings[:, None], Y])
R = np.hstack([(my_ratings > 0)[:, None], R])

Ynorm, Ymean = normalizeRatings(Y, R)

num_movies, num_users = Y.shape
num_features = 7

X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

initial_parameters = np.concatenate([X.ravel(), Theta.ravel()])

options = {'maxiter': 100}

lambda_ = 10
res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,
                                               num_movies, num_features, lambda_),
                        initial_parameters,
                        method='TNC',
                        jac=True,
                        options=options)
theta = res.x

X = theta[:num_movies*num_features].reshape(num_movies, num_features)
Theta = theta[num_movies*num_features:].reshape(num_users, num_features)

/tmp/ipykernel_51156/3937859778.py:17: DeprecationWarning: 'maxiter' has been deprecated in favor of 'maxfun' and will be removed in SciPy 1.11.0.
  res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,


In [25]:
p = np.dot(X, Theta.T)
my_predictions = p[:, 0] + Ymean

movieList = loadMovieList()

ix = np.argsort(my_predictions)[::-1]

print('Top recommendations for you:')
print('----------------------------')
for i in range(15):
    j = ix[i]
    print('Predicting rating %.1f for movie %s' % (my_predictions[j], movieList[j]))

print('\nOriginal ratings provided:')
print('--------------------------')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print('Rated %d for %s' % (my_ratings[i], movieList[i]))

Top recommendations for you:
----------------------------
Predicting rating 5.0 for movie They Made Me a Criminal (1939)
Predicting rating 5.0 for movie Marlene Dietrich: Shadow and Light (1996)
Predicting rating 5.0 for movie Star Kid (1997)
Predicting rating 5.0 for movie Great Day in Harlem, A (1994)
Predicting rating 5.0 for movie Aiqing wansui (1994)
Predicting rating 5.0 for movie Someone Else's America (1995)
Predicting rating 5.0 for movie Entertaining Angels: The Dorothy Day Story (1996)
Predicting rating 5.0 for movie Santa with Muscles (1996)
Predicting rating 5.0 for movie Saint of Fort Washington, The (1993)
Predicting rating 5.0 for movie Prefontaine (1997)
Predicting rating 4.6 for movie Pather Panchali (1955)
Predicting rating 4.5 for movie Shawshank Redemption, The (1994)
Predicting rating 4.5 for movie Schindler's List (1993)
Predicting rating 4.5 for movie Everest (1998)
Predicting rating 4.5 for movie Some Mother's Son (1996)

Original ratings provided:
------------